In [1]:
import pandas as pd

### df 읽고 처리하기

In [2]:
# 파일 읽기
olym_df = pd.read_csv("올림픽대로_통행속도_기상.csv", encoding='cp949')
gaepo_df = pd.read_csv("개포로_통행속도_기상.csv", encoding='cp949')
nonhyun_df = pd.read_csv("논현로_통행속도_기상.csv", encoding='cp949')
dogog_df = pd.read_csv("도곡로_통행속도_기상.csv", encoding='cp949')

# 결측치 확인
print('올림픽대로 결측치 수 : ',olym_df['속도'].isna().sum())
print('개포로 결측치 수 : ',gaepo_df['속도'].isna().sum())
print('논현로 결측치 수 : ',nonhyun_df['속도'].isna().sum())
print('도곡로 결측치 수 : ',dogog_df['속도'].isna().sum())

/var/folders/0t/0jprd7f95qsb1p_xmw35n4l80000gn/T/ipykernel_35545/3343547820.py:3: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  gaepo_df = pd.read_csv("개포로_통행속도_기상.csv", encoding='cp949')


올림픽대로 결측치 수 :  1279
개포로 결측치 수 :  825
논현로 결측치 수 :  1047
도곡로 결측치 수 :  660


In [3]:
def fill_missing_values(df):
    # 일시 컬럼을 datetime 형식으로 변환
    df['일시'] = pd.to_datetime(df['일시'])
    
    # 시간 추출
    df['hour'] = df['일시'].dt.hour
    
    # 각 결측치에 대해
    null_indices = df[df['속도'].isnull()].index
    
    for idx in null_indices:
        current_row = df.loc[idx]
        
        # 현재 결측치의 조건과 동일한 데이터 필터링
        same_conditions = (
            (df['링크아이디'] == current_row['링크아이디']) & 
            (df['hour'] == current_row['hour']) & 
            (df['isHoliday'] == current_row['isHoliday']) &
            (df['속도'].notnull())  # 결측치가 아닌 데이터만
        )
        
        # 조건에 맞는 평균 속도 계산
        mean_speed = df[same_conditions]['속도'].mean()
        
        # 만약 조건에 맞는 데이터가 없다면 (mean_speed가 NaN이라면)
        # 휴일 조건을 제외하고 다시 계산
        if pd.isna(mean_speed):
            same_conditions = (
                (df['링크아이디'] == current_row['링크아이디']) & 
                (df['hour'] == current_row['hour']) &
                (df['속도'].notnull())
            )
            mean_speed = df[same_conditions]['속도'].mean()
        
        # 여전히 평균을 구할 수 없다면 전체 링크아이디의 해당 시간대 평균 사용
        if pd.isna(mean_speed):
            same_conditions = (
                (df['hour'] == current_row['hour']) &
                (df['속도'].notnull())
            )
            mean_speed = df[same_conditions]['속도'].mean()
        
        # 결측치 채우기
        df.loc[idx, '속도'] = mean_speed
        
    return df

# 사용 예시:
olym_df = fill_missing_values(olym_df.copy())
gaepo_df = fill_missing_values(gaepo_df.copy())
nonhyun_df = fill_missing_values(nonhyun_df.copy())
dogog_df = fill_missing_values(dogog_df.copy())

print('올림픽대로 결측치 수 : ',olym_df['속도'].isna().sum())
print('개포로 결측치 수 : ',gaepo_df['속도'].isna().sum())
print('논현로 결측치 수 : ',nonhyun_df['속도'].isna().sum())
print('도곡로 결측치 수 : ',dogog_df['속도'].isna().sum())

올림픽대로 결측치 수 :  0
개포로 결측치 수 :  0
논현로 결측치 수 :  0
도곡로 결측치 수 :  0


## 이상치 파악하기

### 올림픽대로

In [9]:
gaepo_df['기능유형구분']

0         기타도로
1         기타도로
2         기타도로
3         기타도로
4         기타도로
          ... 
525955    기타도로
525956    기타도로
525957    기타도로
525958    기타도로
525959    기타도로
Name: 기능유형구분, Length: 525960, dtype: object

In [7]:
gaepo_df['링크아이디'].unique()

array([1220008200, 1220020500, 1220020400, 1220018700, 1220017300,
       1220015700, 1220012100, 1220008300, 1220013700, 1220012000,
       1220013600, 1220015600, 1220017200, 1220018600, 1220007000])

In [37]:
# 시간대별 속도의 1분위수(25%) 계산
speed_quantiles = olym_df.groupby('hour')['속도'].transform(lambda x: x.quantile(0.25))

# 각 시간대별 1분위수 이하인 데이터만 필터링
ideal_olym = olym_df[olym_df['속도'] <= speed_quantiles][['링크아이디','속도', '일시', 'hour']].sort_values('속도', ascending=True)
ideal_olym.groupby('일시').count().sort_values('링크아이디', ascending=False).head(30)
ideal_olym.head(1000)

,링크아이디,속도,일시,hour
264744,1220005400,6.25,2024-10-15 09:00:00,9
114388,1220005400,6.25,2022-08-19 18:00:00,18
107679,1220005400,6.33,2022-07-15 19:00:00,19
270779,1220005400,6.50,2024-11-15 19:00:00,19
104792,1220005400,6.67,2022-06-30 19:00:00,19
...,...,...,...,...
119758,1220005600,11.42,2022-09-16 17:00:00,17
254581,1220005700,11.42,2024-08-21 19:00:00,19
269423,1220005600,11.42,2024-11-08 17:00:00,17
250808,1220004500,11.42,2024-07-30 19:00:00,19


### 올림픽대로 외 다른 데이터 이상치 확인

In [33]:
# 도곡로 속도 데이터 정렬
ideal_dogog = dogog_df[['링크아이디','속도', '일시', 'hour']].sort_values('속도', ascending=False)
print('\t\t<도곡로 속도 순위>\n',ideal_dogog.head(10))

# 개포로 속도 데이터 정렬
ideal_gaepo = gaepo_df[['링크아이디','속도', '일시', 'hour']].sort_values('속도', ascending=False)
print('\n\t\t<개포로 속도 순위>\n',ideal_gaepo.head(10))

# 논현로 속도 데이터 정렬
ideal_nonhyun = nonhyun_df[['링크아이디','속도', '일시', 'hour']].sort_values('속도', ascending=False)
print('\n\t\t<논현로 속도 순위>\n',ideal_nonhyun.head(10))

		<도곡로 속도 순위>
              링크아이디     속도                  일시  hour
91633   1220025700  58.36 2021-11-15 04:00:00     4
114113  1220025700  58.02 2022-02-01 05:00:00     5
197339  1220025700  57.79 2022-11-17 04:00:00     4
246855  1220025700  57.74 2023-05-08 03:00:00     3
126786  1220025700  56.20 2022-03-17 05:00:00     5
138006  1220025700  56.16 2022-04-25 04:00:00     4
108633  1220025700  56.00 2022-01-13 04:00:00     4
14748   1220025700  55.83 2021-02-21 05:00:00     5
56209   1220025700  55.78 2021-07-15 04:00:00     4
216942  1220025700  55.56 2023-01-24 06:00:00     6

		<개포로 속도 순위>
              링크아이디     속도                  일시  hour
21272   1220018600  49.10 2021-03-01 02:00:00     2
15566   1220012100  47.36 2021-02-13 05:00:00     5
81090   1220018600  46.35 2021-08-14 06:00:00     6
76743   1220018600  45.94 2021-08-02 04:00:00     4
400371  1220018700  45.37 2024-01-18 03:00:00     3
391752  1220018700  45.22 2023-12-25 04:00:00     4
397510  1220018700  44.75 2024-01

In [24]:
dogog_df.keys()

Index(['일자', '요일', '도로명', '링크아이디', '시점명', '종점명', '방향', '거리', '차선수', '기능유형구분',
       '도심/외곽구분', '권역구분', '시간', '속도', '일시', '지점', '지점명', '기온(°C)', '강수량(mm)',
       '풍속(m/s)', '풍향(16방위)', '적설(cm)', 'seq', 'dateKind', 'isHoliday',
       'dateName', 'hour'],
      dtype='object')

In [26]:
dogog_df[['링크아이디', '시점명', '종점명','권역구분','지점', '지점명']]

,링크아이디,시점명,종점명,권역구분,지점,지점명
0,1220020800,한티역,영동세브란스,강남구,108,서울
1,1220016700,싸리고개공원입구,도곡1동주민센터,강남구,108,서울
2,1220015400,싸리고개공원입구,뱅뱅사거리,강남구,108,서울
3,1220016600,도곡1동주민센터,싸리고개공원입구,강남구,108,서울
4,1220019000,영동세브란스,도곡1동주민센터,강남구,108,서울
...,...,...,...,...,...,...
420763,1220020800,한티역,강남세브란스,강남구,108,서울
420764,1220019000,강남세브란스,도곡1동주민센터,강남구,108,서울
420765,1220015400,싸리고개공원입구,뱅뱅사거리,강남구,108,서울
420766,1220025700,대치동우성아파트,탄천2교,강남구,108,서울


In [34]:
# 도곡로 기능유형구분 확인
print('\t\t<도곡로 기능유형구분>\n')
print(dogog_df['기능유형구분'].unique())

# 개포로 기능유형구분 확인 
print('\n\t\t<개포로 기능유형구분>\n')
print(gaepo_df['기능유형구분'].unique())

# 논현로 기능유형구분 확인
print('\n\t\t<논현로 기능유형구분>\n') 
print(nonhyun_df['기능유형구분'].unique())


		<도곡로 기능유형구분>

['보조간선도로']

		<개포로 기능유형구분>

['기타도로']

		<논현로 기능유형구분>

['주간선도로']
